# Tensorflow Basics
*By Ronny Restrepo*

# Graph and Session 


For this tutorial I am assuming that you are already have some familiarity with artificial neural networks, matrices, vectors and feel comfortable with Python, and numpy. I am also assuming that you have installed Tensorflow. If not then please head over to the [official tensorflow installation guide](/home/ronny/programming/machine_learning/tests/spatial_transformers/final_digit_pos_scale_testB) and follow the instructions for your operating system. 


## Template

All the examples in this tutorial will build off of the template in the following cell. Run this code to have the relevant libraries loaded up. 

In [2]:
# ================================================
#                              TENSORFLOW TEMPLATE
# ================================================
from __future__ import print_function, division
import tensorflow as tf
import numpy as np

# ------------------------------------------------
#                                    Build a graph
# ------------------------------------------------
graph = tf.Graph()
with graph.as_default():
    pass # REPLACE WITH YOUR CODE TO BUILD A GRAPH

# ------------------------------------------------
#              Create a session, and run the graph
# ------------------------------------------------
with tf.Session(graph=graph) as session:
    pass # REPLACE WITH YOUR CODE TO RUN A SESSION

You will notice that there are two main sections of code involved. One for 
building a graph and one for running a session. Before continuing with any 
code, it is important to understand what is meant by these two things. 

## Graph 
A graph is a set of operations that are connected together in some way. For 
example, the diagram below, shows the graph representation of this formula: 

$$y=\frac{(A + B)}{3} \times \frac{C}{2}$$

We see the inputs **A**, **B** and **C**. Their values travel along the green 
lines (called the *edges of the graph*). And on the blue nodes, we apply some 
operation to those values coming in. 

![image of graph](TUTimages/tf_graph_and_session/graph_diagram1.png)

The values that travel along the edges of the graph are *tensors* (hence the name *Tensorflow*). Tensors can be thought of as a generalization of scalar values, vectors and matrices, such that: 

- **0 Dimensional Tensor : ** is equivalent to a scalar value (eg a float or an integer)

- **1 Dimensional Tensor : ** is equivalent to a vector, eg $$\begin{bmatrix}
    1.2 \\ 
    3.4 \\ 
    5.6
    \end{bmatrix}$$
    
- **2 Dimensional Tensor : ** is equivalent to a matrix, eg: 
$$
\begin{bmatrix}
1.2 &  2.2&  3.3 & 4.4\\ 
3.4 &  4.2&  6.7 & 3.3\\ 
5.6 &  5.3&  5.6 & 4.4
\end{bmatrix}
$$

- **N Dimensional Tensor : ** is extends the concept of a matrix to N dimensions.  


In the context of Neural Networks, the graph can be thought of as the architecture of our neural network. The inputs will be things like our training data and labels. The operations will be things like matrix dot products, non-linearity functions, loss functions, etc.

Specifying the graph that is shown in the diagram above can be done as follows. Run the code in the following cell to create the graph. 

In [3]:
# ------------------------------------------------
#                                    Build a graph
# ------------------------------------------------
graph = tf.Graph()
with graph.as_default():
    A = tf.constant(3.0)           # Create a constant value
    B = tf.constant(9.0)
    C = tf.constant(4.0)
    
    summed = tf.add(A, B)          # Adition Operation 
    div3 = tf.div(summed, 3.0)     # Division Operation
    
    div2 = tf.div(C, 2.0)          # Division Operation
    
    mult = tf.mul(div3, div2) # Multiplication operation

Now, lets try get the value of the `mult` operation in the graph by running the following code.

In [ ]:
print(mult)

You might have expected it to output a value such as `8.0`, which would be the 
value of running the operations specified, but instead you get a Tensor object.

The reason is that when you specify a tensorflow graph, tensorflow does not 
actually run the operations, it simply creates a specification for how data 
should flow. In order to actually make the data flow through the graph, we need 
to initialize and run a session. This will be explained in the following section. 

## Session
In order to actually run data through a Tensorflow graph we need to initialize 
a session and specify what portion of the graph we want to run. In order to initialize a session we run the `tf.Session()` function, passing as an argument the graph that we wish to use. Once the session is initialized, we tell it what portion of the graph to run by calling the `run()` method. 

In the following cell of code, we initialize a session and name it `sess`. We then call `sess.run()` to run the operations in the graph. 

Run the following code to see the results. 

In [17]:
# ------------------------------------------------
#              Create a session, and run the graph
# ------------------------------------------------
with tf.Session(graph=graph) as sess:
    mult_output = sess.run(mult)
    print(mult_node_output)

8.0


When calling `sess.run()` we pass to it the operation that we are interested in running from the graph. Tensorflow automatically runs any other operations which your desired operation depends on. 

So for instance, if you specified that you want to run the `div3` operation (as in the following cell of code), then it will automatically also run the `summed` operation, but it will not compute the `div2` or `mult` operations. 

In [12]:
# ------------------------------------------------
#              Create a session, and run the graph
# ------------------------------------------------
with tf.Session(graph=graph) as sess:
    summed_output = sess.run(summed)
    print(summed_output)

12.0


![image of graph](TUTimages/tf_graph_and_session/graph_diagram_div3.png)

### Returned Values from sess.run()

The first argument that we pass on to `sess.run()` is an operation that we want to run in the graph. The value that this operation evaluates to is returned by `sess.run()` as a numpy object. If the operation evaluates to a scalar value, then `sess.run()` will return a numpy scalar object like a numpy float. 

Run the following code to see the data type of the value that was returned when the `mult` operation was evaluated. 

In [21]:
type(mult_output)

numpy.float32

If the operation in the graph evaluates to a 1D, 2D, or ND tensor, then `sess.run()` will return a numpy array object. 

### Evaluating/Returning Multiple Operations
If you wish to either evaluate several operations in the graph that are not dependedent on each other, or simply wish to find out the value passing through several parts of the graph, you can pass a list of operations as the first argument to `sess.run()`. 

For example, if you run the following code, you will evaluate the `summed`, `div3` and `div2` operations, and return their evaluated values. 

In [4]:
# ------------------------------------------------
#              Create a session, and run the graph
# ------------------------------------------------
with tf.Session(graph=graph) as sess:
    summed_output, div3_output, div2_output = sess.run([summed, div3, div2])
    print(summed_output)
    print(div3_output)
    print(div2_output)

12.0
4.0
2.0


![image of graph being run to certain ops](TUTimages/tf_graph_and_session/graph_diagram_multi_ops.png)

### Precaution 
**WARNING!: ** be very carefull not override the values of the python variables that refer to tensor operations that you wish to evaluate in the graph. Notice that we stored the returned value of the `mult` operation in a variable called `mult_output`, and not `mult`. Run the following few cells of code to see what happens when you over-ride the variable name. 

In [23]:
# Object type of `mult` prior to over-riding
type(mult)

tensorflow.python.framework.ops.Tensor

In [ ]:
# ------------------------------------------------------------------
# Assign output of sess.run() to same variable name as the tensor op
# ------------------------------------------------------------------
with tf.Session(graph=graph) as sess:
    mult = sess.run(mult)    # overriding `mult`
    print(mult_node_output)

In [ ]:
type(mult)

So far it seems ok... but notice that the data type of `mult` has changed. Now try running the session again, using the same arguments. 

In [ ]:
# ------------------------------------------------
#   Running session again after variable over-ride
# ------------------------------------------------
with tf.Session(graph=graph) as sess:
    mult = sess.run(mult)   # trying to call the `mult` operation
    print(mult_node_output)

You will get a `TypeError` because you are feeding to `sess.run()` an object `mult_node` that is now a numpy object and not a tensor operation object. 

## EXTRA:  Why Tensorflow splits the process into graph and Session
Python is a wonderful programming language. It is simple and fun to use. However, it is not very fast. Performing math operations on large matrices can be time consuming. 

Many Python libraries for scientific computing have been designed to overcome this issue by being written in some other programming language like C or C++. By performing the calculation in a faster programming language, they speed things up. 

You interact with those libraries using using python functions as you normally would. But under the hood, it takes the data and performs the operation outside of python, then returns the data back into Python. 

This greatly speeds things up. However, there is still some overhead involved in transfering the data in and out of Python. The overhead involved in transfering data is even more pronounced when we are making calculations on a distributed system that spread the computations across multiple computers.

Instead of transfering data in and out of Python after every single operation, Tensorflow will run a series of operations outside of Python. Only after all those operations have been performed will it return back to Python. 

This is the point of creating a graph. A graph is a way to specify the series of steps that will need to be calculated. Opening a session creates a connection to the system that will actually perform the calculations. And running a portion of the graph specifies how much of the operations will need to be calculated outside of Python before it returns again. 

In [2]:
# IGNORE THIS CODE: Irrelevant to the tutorial 
# For Prettyfying the notebook fonts and styles when run locally
from IPython.core.display import HTML
HTML("""<link rel="stylesheet" href="./custom.css" type="text/css" />""")